# W207 Final Project: Facial Recognition

### Team: MapReduce, MapReuse, MapRecycle
Members: Tennison Yu, Madeleine Bulkow, Mark Paluta, Alex Dauenhauer

https://github.com/tyu0912/MIDS_W207_facial_recognition/issues

# Introduction

This is the final project submission from Team MapReduce,MapReuse,MapRecycle for W207: Applied Machine Learning as part of the MIDS program at UC Berkeley. We are working with data from https://www.kaggle.com/c/facial-keypoints-detection/data and the goal is to accurately identify facial features on an (x, y) coordinate system based on an input image. 

## Overall Project Flow

Our project is layed out into the following parts:

1) EDA - We explore the data and see what the images and labels we are working with are like

2) Feature Engineering - We alter/augment images to expand our training set to better cover edge cases and generalize the algorithm. Based on our exploration of common standard techniques done, we decided to flip, contrast, blur, and rotate the images.

3) Simple Modeling - We explore the effectiveness of simple algorithms such as Decision Trees and Neast Neighbors and compare and contrast their effectiveness. 

4) Complex Modeling - We explore the effectiveness of a more complex model, convoluted neural networks (CNN).

5) Summary and Apply - We present our findings and apply our best predictive algorithm to the test data that Kaggle provided us to see how well we did. 

# Import Packages, Load data

In [ ]:
# # Optional cell to install/update all required packages
# ! pip install keras tensorflow numpy pandas matplotlib scipy imgaug opencv-python pydot graphviz > nul
# ! conda install shapely > nul
# print('Packages installed, output supressed for cleanliness')

In [ ]:
import time, sys, os, keras, tensorflow as tf, warnings, imgaug as ia
import numpy as np, pandas as pd, matplotlib.pyplot as plt, pydot, graphviz
from importlib import reload
from sklearn.utils import shuffle
from keras import backend as K
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import plot_model
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, mean_squared_error
# from sklearn.model_selection import GridSearchCV
from scipy import stats
from imgaug import augmenters as iaa
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error


# set tensorflow as keras backend
if K.backend() not in ["tensorflow", 'plaidml']:
    print ('Switching keras backend...')
    os.environ['KERAS_BACKEND'] = "tensorflow"
    reload(K)
    
print('keras version:', keras.__version__)
print('tensorflow version:', tf.__version__)
print('pandas version:', pd.__version__)
print('numpy version:', np.__version__)

# Initiate settings and load the data.
np.random.seed(0)
pd.options.display.max_columns = 30
warnings.filterwarnings('ignore')

train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

print ('Everything Loaded Successfully')

# EDA

We begin by exploring our data. It is important to understand the range of values we can take on, whether there is missing data, and whether there might be erroneous data. We will be exploring primarily on the image level, as opposed to on the individual keypoint or pixel level, because pixels and keypoints tell a much more important story via their relationship to one another rather than in isolation.

## Glancing at the Data

First, we will see the format our data is in. They keypoints are provided in x or y coordinate floats.

In [ ]:
train[train.columns[:-1]].head(5)

The pixels are in a greyscale 8-bit format (range 0 to 255).

In [ ]:
train['Image'].head(5)

Our first interesting observation is that not all of the images contain all keypoints. In fact no keypoint is present in every image and many are present in less than half of the images. 

In [ ]:
print(train.count())

From the above cells, we can see that we are given x,y coordinates for our different features in seperate columns. As well, images are currently strings with each pixel value separated by a space. We will need to manipulate these to a form we can work with such as numpy arrays.

Additionally, many labels have a lot of missing data. The only label that is complete appears to be the nose tip.

## Reformating the Data

It seems for complex models such as the CNN, keras requires the data to be in numpy array form with dimensions = (batch, image_width, image_height, num_channels). Images in dataset are 96x96 grayscale images, therefore shape should be (X, 96, 96, 1).

To ensure we have data to work stored away for development, we will also split the training set into 80% train data and 20% dev data in the steps below.

In [ ]:
def process_datasets(df, test=False, fillna=False, dropna=False):
    '''This function reformats the data from a dataframe to a numpy
    array of the appropriate shape.
    '''

    # whether or not to remove images with missing labels
    if dropna:
        df = df.dropna()
    
    # build feature dict for reference in future functions
    feature_dict = {label:i for i,label in enumerate(df.columns[:-1].values)}
    
    # separate images into numpy arrays
    images_pixel_feature = df.Image.apply(lambda im: np.fromstring(im, sep=' '))
    
    # rescale pixel values to [0,1] interval and reshape
    images_pixel_feature = np.stack(images_pixel_feature) / 255
    images_pixel_feature = images_pixel_feature.reshape(-1, 96, 96, 1)
    
    
    # separate the labels and scale them to [-1,1] interval for use of 
    # rectified linear unit activation
    if not test:
        facial_point_labels = df.iloc[:, :-1]
        if fillna:
            facial_point_labels.fillna(np.mean(facial_point_labels, axis=0), inplace=True)
        facial_point_labels = facial_point_labels.values
        facial_point_labels = (facial_point_labels - 48) / 48
    else:
        facial_point_labels = None
    
    return images_pixel_feature, facial_point_labels, feature_dict

In [ ]:
# Raw data
# train_raw = processed but not split
train_data_raw, train_labels_raw, feature_dict = process_datasets(train)
test_data, test_labels, _ = process_datasets(test, test=True)

## Examine Random Sample

Here we wanted to observe some sample images with labels to see what kind of variety of images we are working with. 

In [ ]:
def plot_sample(data, labels, labels2=None):
    '''
    data must be a numpy.ndarray of shape (X, 96, 96, 1)
    labels must be a numpy.ndarray (X, 30)
    X must be any square number
    labels = true labels
    labesl2 = predicted labels (optional)
    '''
    
    dim = np.sqrt(len(data))
    fig = plt.figure(figsize=(10,10))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i in range(len(labels)):
        img = data[i].reshape(96,96)
        ax = fig.add_subplot(dim, dim, i + 1, xticks=[], yticks=[])
        ax.imshow(data[i].reshape(96,96), cmap='gray')
        ax.scatter(labels[i][0::2] * 48 + 48, labels[i][1::2] * 48 + 48, c='r')
        if not labels2 is None:
            ax.scatter(labels2[i][0::2] * 48 + 48, labels2[i][1::2] * 48 + 48, c='b')

In [ ]:
# Output sample of random images. Some of these will be the NA data
n = 25
idx = np.random.randint(0, len(train_data_raw), n)
data_sample = train_data_raw[idx, :, :, :]
label_sample = train_labels_raw[idx, :]
plot_sample(data_sample, label_sample)

## Examine Edge Cases

In addition to random faces, we wondered what some edge cases are like. 

In [ ]:
edge_cases = []

# --- compile interesting cases --- 

# noses on the side of the image may imply a profile shot
edge_cases.extend(train_labels_raw[:,feature_dict['nose_tip_x']].argsort()[:8])           # left profiles
edge_cases.extend(train_labels_raw[:,feature_dict['nose_tip_x']].argsort()[-8:])          # right profiles

# max differential in eye height will imply clocking
edge_cases.extend((train_labels_raw[:,feature_dict['right_eye_center_y']] -               # faces clocked right
                   train_labels_raw[:,feature_dict['left_eye_center_y']]).argsort()[:8])  
edge_cases.extend((train_labels_raw[:,feature_dict['left_eye_center_y']] -                # faces clocked left
                   train_labels_raw[:,feature_dict['right_eye_center_y']]).argsort()[:8]) 

# faces vary in width
edge_cases.extend((train_labels_raw[:,feature_dict['right_eye_center_x']] -               # wide faces
                   train_labels_raw[:,feature_dict['left_eye_center_x']]).argsort()[:8])  
edge_cases.extend((train_labels_raw[:,feature_dict['left_eye_center_x']] -                # thin faces
                   train_labels_raw[:,feature_dict['right_eye_center_x']]).argsort()[:8]) 

# investigate very dark or light faces to see if we have different races or lighting
edge_cases.extend(np.sum(train_data_raw.reshape(train_data_raw.shape[0], 96*96), axis=1).argsort()[:8])      # dark faces
edge_cases.extend(np.sum(train_data_raw.reshape(train_data_raw.shape[0], 96*96), axis=1).argsort()[-8:])     # light faces

print(len(edge_cases), edge_cases)

data_edge = train_data_raw[edge_cases, :, :, :]
label_edge = train_labels_raw[edge_cases, :]

The rows of the following image are meant to illustrate, respectively:
- left profiles
- right profiles
- clocked right
- clocked left
- wide faces
- thin faces
- dark faces
- light faces

In [ ]:
plot_sample(data_edge, label_edge)

From the above, it looks like there are several different types of edge cases. We can see there are a variety of angles, lighting, tilts, resolutions, and even a drawing (bottom right corner). Some cases look like purely bad data that may need to be removed such as the first picture of Leonardo DiCaprio. Also interesting is that there are images that are the same subject.

### Removing erroneous data  
To remove the data that we considered bad data, we utilized a method that will be covered in our appendix to flag likely erroneous images. They are printed here to confirm that they indeed look erroneous, and then we throw them out for the remainder of our project.

In [ ]:
# hard code in flagged images
flagged = [1230, 1620, 1649, 1652, 1723, 1747, 1808, 1834, 1861, 1877, 1881, 1907, 1927, 1942, 1966, 1995,
           2036, 2096, 2175, 2191, 2199, 2244, 2289, 2453, 2484, 2533, 2562, 2629, 2664, 2700, 2787, 2818,
           2831, 2845, 2982, 3087, 3173, 3296, 3307, 3374, 3447, 3487, 3510, 3647, 3888, 4180, 4263, 4480,
           4482, 4490, 4601, 4717, 4786, 5117, 5795, 5952, 6082, 6271, 6315, 6405, 6407, 6492, 6493, 6569,
           6585, 6765, 6782, 6834, 6906]

# plot these images as a sanity check that they are in fact "bad" data
i = 0
rows = 8
cols = len(flagged)//rows + 1
fig = plt.figure(figsize=(20,20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for index in flagged:
    ax = fig.add_subplot(rows, cols, i + 1, xticks=[], yticks=[])
    ax.imshow(train_data_raw[index].reshape(96,96), cmap='gray')
    ax.scatter(train_labels_raw[index][0::2] * 48 + 48, train_labels_raw[i][1::2] * 48 + 48, c='r')
    i += 1

In [ ]:
# Delete these images from the data set forever
train_data_raw = np.delete(train_data_raw, flagged, 0)
train_labels_raw = np.delete(train_labels_raw, flagged, 0)

Because there are so many images with incomplete values, we now need to define another set with NA removed that will be used later on to train simple baseline models as well as predict the position of the missing labels for later training

In [ ]:
# Dropping NAs
# drop bad data from original dataframe
train = train.drop(flagged)

# train_complete = no missing values
train_data_complete, train_labels_complete, _ = process_datasets(train, dropna=True)

In [ ]:
### DON'T RUN THIS CELL MORE THAN ONCE WITHOUT RE-RUNNING PROCESS DATASETS! ###

# 20% of the data will go to dev
percent_to_dev = 0.2 

# Note the naming convention change.
# train_orig = with missing values

train_data_orig, dev_data_orig, train_labels_orig, dev_labels_orig = train_test_split(
    train_data_raw, train_labels_raw, test_size=percent_to_dev
)
train_data_complete, dev_data_complete, train_labels_complete, dev_labels_complete = train_test_split(
    train_data_complete, train_labels_complete, test_size=percent_to_dev
)

print("\nTrain data shape", train_data_orig.shape)
print("Dev data shape", dev_data_orig.shape)
print("Train labels shape", train_labels_orig.shape)
print("Dev labels shape", dev_labels_orig.shape)
print("\nTrain data no-missing-values shape", train_data_complete.shape)
print("Dev data no-missing-values shape", dev_data_complete.shape)
print("Train labels no-missing-values shape", train_labels_complete.shape)
print("Dev labels no-missing-values shape", dev_labels_complete.shape)

print("\nData Generation Finished")

# Feature Engineering

To be sure our training set has enough variety to predict well on the observed edge cases, we expand the variety of our data set even more. We decided to create sets of images that were mirrored, altered contrast, blurred, and rotated as it seems these are the most prevalent gestures of the different faces. 

## Perform Augmentations

In [ ]:
# We use the imgaug library to generate our new sets of images

def augment_images(data, labels, aug):
    
    data = data * 255
    aug_labels =[]
    seq_det = aug.to_deterministic()
    aug_data = seq_det.augment_images(data)
    
    keypoints_on_images = []
    
    for features in labels:
        keypoints = []
        
        for c in range(0, features.shape[0], 2):
            x = features[c]* 48 + 48
            y = features[c+1]* 48 + 48

            keypoints.append(ia.Keypoint(x=x, y=y))

        keypoints_on_images.append(ia.KeypointsOnImage(keypoints, shape=(96,96)))
    
    keypoints_aug = seq_det.augment_keypoints(keypoints_on_images)    
        
    for keypoints_after in keypoints_aug:
        aug_labels_set = []
        
        for i, keypoint in enumerate(keypoints_after.keypoints):
            aug_labels_set.append((keypoint.x - 48) / 48)
            aug_labels_set.append((keypoint.y - 48) / 48)
                
        aug_labels.append(aug_labels_set)
        
    return aug_data/255, np.array(aug_labels)

# The imgaug library confuses the labels in the case of mirroring an image so we made our own custom method. 

def mirror_data(data, labels, features):
    '''This function flips the images and labels to their new columns
    Input data is numpy array of shape (X, 96, 96, 1)
    Input labels is numpy array of shape (X, 30)
    features is a list of column header strings
    '''
    
    # flip the images
    data_flipped = data[:, :, ::-1, :]
    
    # flip the labels
    labels_flipped = np.zeros(labels.shape)
    for idx in range(len(labels)):
        for i,s1 in enumerate(features):
            parts = s1.partition('_')
            if parts[0] == 'left' or parts[0] == 'right':
                coord = parts[-1]
            elif parts[0] == 'mouth':
                parts = s1.split('_', maxsplit=2)
                if 'corner' in parts[-1]:
                    coord = parts[-1]        
                elif 'x' in parts[-1]:
                    labels_flipped[idx,i] = labels[idx,i] * -1
                else:
                    labels_flipped[idx,i] = labels[idx,i]
            else:
                if 'x' in parts[-1]:
                    labels_flipped[idx,i] = labels[idx,i] * -1
                else:
                    labels_flipped[idx,i] = labels[idx,i]
            for j in range(i+1,len(features)-1):
                s2 = features[j]
                if coord in s2:
                    if 'x' in coord:
                        labels_flipped[idx,i] = labels[idx,j] * -1
                        labels_flipped[idx,j] = labels[idx,i] * -1
                    else:
                        labels_flipped[idx,i] = labels[idx,j]
                        labels_flipped[idx,j] = labels[idx,i]
                
    return data_flipped, labels_flipped

In [ ]:
# mirror training and dev data
flipped_data, flipped_labels = mirror_data(
    train_data_orig, train_labels_orig, list(feature_dict.keys())
)

flipped_data_complete, flipped_labels_complete = mirror_data(
    train_data_complete, train_labels_complete, list(feature_dict.keys())
)

# Concatanating to make new set

# Note the new naming convention
# train_aug = augmented with missing values  
# train_aug_comp = augmented no missing values

train_data_aug = np.concatenate((train_data_orig,flipped_data), axis=0)
train_labels_aug =  np.concatenate((train_labels_orig, flipped_labels), axis=0)

train_data_aug_comp = np.concatenate((train_data_complete, flipped_data_complete),axis=0)
train_labels_aug_comp = np.concatenate((train_labels_complete,flipped_labels_complete),axis=0)

In [ ]:
# We test applying 1,2 or 3 of blurring, contrast to the data. 
# We do this randomly to make sure everything is equally distributed.

aug_test = iaa.Sometimes(1,  iaa.SomeOf((1, 3), 
            [iaa.GammaContrast((0.25, 3)),
             iaa.GaussianBlur(sigma=1),
             iaa.Affine(rotate=(-10,10), scale={"x": (1.05, 1.2), "y": (1.05, 1.2)})
            ]                        
        )
    )

fig, axes = plt.subplots(1,6,figsize=(20,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)


for i in range(6):
    test_data_aug, test_labels_aug = augment_images(np.expand_dims(train_data_aug_comp[0], axis=0), np.expand_dims(train_labels_aug_comp[0], axis=0), aug_test)
    ax = axes[i]
    
    if i == 0:
        ax.set_title('Original', fontsize=20)
        ax.imshow(train_data_aug_comp[0].reshape(96,96), cmap='gray')
    else:
        ax.set_title('Sample Aug {i}'.format(i=i), fontsize=20)
        ax.imshow(test_data_aug.reshape(96,96), cmap='gray')
    
    ax.scatter(test_labels_aug[0][0::2] * 48 + 48, test_labels_aug[0][1::2] * 48 + 48, c='r')
    ax.axis('off')
    

In [ ]:
# We apply the above to to 50% of the data so that there
# is an even amount of unmanipulated and manipulated data.

aug = iaa.Sometimes(0.5,  iaa.SomeOf((1, 3), 
            [iaa.GammaContrast((0.25, 3)),
             iaa.GaussianBlur(sigma=1),
             iaa.Affine(rotate=(-10,10), scale={"x": (1.05, 1.2), "y": (1.05, 1.2)})
            ]                        
        )
    )


train_data_aug, train_labels_aug = augment_images(train_data_aug, train_labels_aug, aug)
train_data_aug_comp, train_labels_aug_comp = augment_images(train_data_aug_comp, train_labels_aug_comp, aug)

## Check Data Augmentation

In [ ]:
img1 = train_data_orig[0].reshape(96,96)
lab1 = train_labels_orig[0]
img2 = train_data_aug[0].reshape(96,96)
lab2 = train_labels_aug[0]
img3 = train_data_orig[1].reshape(96,96)
lab3 = train_labels_orig[1]
img4 = train_data_aug[1].reshape(96,96)
lab4 = train_labels_aug[1]
img5 = train_data_orig[2].reshape(96,96)
lab5 = train_labels_orig[2]
img6 = train_data_aug[2].reshape(96,96)
lab6 = train_labels_aug[2]

In [ ]:
imgs = [img1, img2, img3, img4, img5, img6]
labs = [lab1, lab2, lab3, lab4, lab5, lab6]
titles = ['original 1','augmented 1','original 2','augmented 2','original 3', 'augmented 3']

fig, axes = plt.subplots(1,6,figsize=(20,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(6):
    ax = axes[i]
    ax.imshow(imgs[i], cmap='gray')
    ax.scatter(labs[i][0::2] * 48 + 48, labs[i][1::2] * 48 + 48, c='r')
    ax.axis('off')
    ax.set_title(titles[i], fontsize=20)

It seems that the augmentations worked. Note again that the augmentations are applied to only 50% of the data so there may be some images above that are not augmented. 

# Establish Baseline

To begin our modeling, we select common supervised-learning regression algorithms such as linear regression, k-nearest neighbors, decision tree and random forest for baselines to compare our CNN to. The thought process behind choosing each one is described below. However, given existing literature, it seems CNN is the preferred way to solve problems of this nature. Therefore greater emphasis will be given for that technique. 

**Kaggle Output**

In [ ]:
def predict_for_kaggle(model, test_data, filename):
    
    #Predict values for your chosen model
    try:
        pred = model.predict(test_data)
    except:
        pred = model
    
    # Get the list of required test points
    lookup = pd.read_csv('IdLookupTable.csv')
    
    #Match up the test points with the predicted values
    labelNums = {}
    for index, label in enumerate(train.columns.values):
        labelNums[label] = index
    lookup['FeatureNum'] = lookup.apply(
        lambda row: labelNums[row['FeatureName']], axis=1
    )
    lookup.Location =  np.minimum(
        96, np.maximum(0, pred[lookup.ImageId-1, lookup.FeatureNum]*48 + 48)
    )
    
    # save the rowid and the associated location to a csv
    lookup.to_csv(
        path_or_buf=filename, columns=['RowId','Location'], index=False
    )
    
    return pred

## Simple Models

To begin our modeling, in a similar fashion to Shi (2017), we select common supervised-learning regression algorithms such as linear regression, k-nearest neighbors, decision tree and random forest for baselines to compare our CNN to. The thought process behind choosing each one is described below. However, given existing literature, it seems CNN is the preferred way to solve problems of this nature. Therefore greater emphasis will be given for that technique. The choice of these algorithms is also partially inspired by https://scikit-learn.org/stable/auto_examples/plot_multioutput_face_completion.html#sphx-glr-auto-examples-plot-multioutput-face-completion-py 

We start off by seeing how different the average location is for each feature versus the different images themselves. This gives a "no work done" baseline to compare future results to.

In [ ]:
# As a simple test, we want to look at the average of all the features and what the error of that would be. 
# We use the complete set to avoid issues with incomplete values. 

avg_dict = {}

avg_position = np.mean(train_labels_aug_comp, axis=0)
predictions = np.ones(train_labels_aug_comp.shape) * avg_position
mse = np.sqrt(mean_squared_error(train_labels_aug_comp, predictions)) * 48
avg_dict['Training Average'] = [None, None, None, predictions]

predict_for_kaggle(predictions, test_data, 'average_result_kaggle.csv')

print ('The RMSE error when predicting average value for each label:', mse)

The following code will model the data using the chosen baseline models. The models will run pre-optimized as the grid search portions of the code were done prior to assignment submission. 

In [ ]:
# More simple baselines
regressors = {
    'LinearRegression': LinearRegression(normalize = True),
    'DecisionTree': DecisionTreeRegressor(random_state=0, min_samples_split=2),
    'RandomForest': RandomForestRegressor(random_state=0, min_samples_split=2, n_estimators=6),
    'NearestNeighbors': KNeighborsRegressor(weights='uniform', metric='euclidean')
} 

# Optimization Parameters

parameters = {
    'RandomForest': {'n_estimators': [6,8,10,12]}, 
    'DecisionTree': {'min_samples_split': [2,3,4]},
    'NearestNeighbors':  {'weights': ['uniform','distance'], 'metric': ['euclidean','manhattan','minkowski']}, 
    'LinearRegression': {'fit_intercept': [True, False], 'normalize':[True, False]}
}

rmse_scorer = make_scorer(mean_squared_error)

def model_training(dict_name, model, parameters, train_data, labels, dev_data, dev_labels, test_data, grid = False):
    X = train_data.reshape(-1, 96**2)
    Y = labels
    
    if grid == True:        
        grid_obj = GridSearchCV(model, parameters, scoring=rmse_scorer)
        grid_obj = grid_obj.fit(X, Y)
        
        print(grid_obj.best_params_)
        
        regressor = grid_obj.best_estimator_
        start=datetime.now()
        regressor.fit(X, Y)
        runtime = datetime.now() - start
        predictions = regressor.predict(X)
        
        train_mse = np.sqrt(mean_squared_error(Y, predictions)) * 48

        Z = dev_data.reshape(-1, 96**2)

        predictions = regressor.predict(Z)
        dev_mse = np.sqrt(mean_squared_error(dev_labels, predictions)) * 48
        
        predict_for_kaggle(regressor, test_data.reshape(-1, 96**2), dict_name +'_kaggle.csv')
    
    else:
    
        start=datetime.now()
        model.fit(X,Y)
        runtime = datetime.now() - start

        predictions = model.predict(X)
        train_mse = np.sqrt(mean_squared_error(Y, predictions)) * 48

        Z = dev_data.reshape(-1, 96**2)

        predictions = model.predict(Z)
        dev_mse = np.sqrt(mean_squared_error(dev_labels, predictions)) * 48
        
        predict_for_kaggle(model, test_data.reshape(-1, 96**2), dict_name +'_kaggle.csv')
    
    return [round(train_mse,2), round(dev_mse,2), round(runtime.seconds,2), predictions]


In [ ]:
def train_basic_models(which_set, avg_dict, train_data, train_labels, dev_data, dev_labels, test_data, grid = False):
    if which_set == 'complete':
        print ('This is with the complete set')
    elif which_set == 'complete_aug':
        print ('This is with the complete_aug set')
    
    basic_models = {}
    
    for k,v in regressors.items():
        p = parameters[k]
        
        if which_set == 'complete':
            dict_name = k + '_complete'
        elif which_set == 'complete_aug':
            dict_name = k + '_complete_aug'
        
        result = model_training(dict_name, v, p, train_data, train_labels, dev_data, dev_labels, test_data, grid)
        
        basic_models[dict_name] = result
        
        print('\n', k,':\n',
              'Training RMSE=', result[0],
              'Dev RMSE=', result[1],
              'Train Time=', result[2]
             )

    img_idx = 2
    img = train_data_complete[img_idx].reshape(96,96)

    fig, axes = plt.subplots(1,len(basic_models) + 1,figsize=(15,15))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    print ('\nResults Below. Red = Actual Labels  Blue = Predicted Labels\n')
    
    ax = axes[0]
    ax.imshow(img, cmap='gray')
    ax.scatter(train_labels_complete[img_idx][0::2] * 48 + 48, train_labels_complete[img_idx][1::2] * 48 + 48, c='r')
    ax.scatter(avg_dict['Training Average'][3][img_idx][0::2] * 48 + 48, avg_dict['Training Average'][3][img_idx][1::2] * 48 + 48, c='b')
    ax.axis('off')
    ax.set_title('Training Average', fontsize=11)
    
    for i,(k,v) in enumerate(basic_models.items()):
        ax = axes[i+1]
        ax.imshow(img, cmap='gray')
        ax.scatter(train_labels_complete[img_idx][0::2] * 48 + 48, train_labels_complete[img_idx][1::2] * 48 + 48, c='r')
        ax.scatter(v[3][img_idx][0::2] * 48 + 48, v[3][img_idx][1::2] * 48 + 48, c='b')
        ax.axis('off')
        ax.set_title(k, fontsize=15)
        
    return basic_models

In [ ]:
results_comp = train_basic_models('complete', avg_dict, train_data_complete, train_labels_complete, dev_data_complete, dev_labels_complete, test_data)

In [ ]:
results_comp_aug = train_basic_models('complete_aug', avg_dict, train_data_aug_comp, train_labels_aug_comp, dev_data_complete, dev_labels_complete, test_data)

In [ ]:
def merge_dicts(*dict_args):
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

result_dict = merge_dicts(avg_dict, results_comp, results_comp_aug)

### Overall Results

Without augmentations, it seems that kNN performs the best followed by Random Forest and Linear Regression. This isn't surprising as kNN essentially assigns the k points by it's closest values. As supplemented below, all facial features are approximately at the same location on the face. With Decision Trees and Random Forests, a single Decision Tree performs the worst but after aggregating to a Random Forest, the algorithm performs a lot better. Random Forests typically reduce variance by training on different samples of the data. Another way that it reduces overfitting is that it uses a random subset of features each time. This allows the overall algorithm to generalize better.

When looking at the augmented result, kNN and Random Forest retain their performance ranking however linear regression performs much more poorly. This is likely to do with the manipulated images being introduced. We elaborate further on its weakness below. 

### Linear Regression

**Rational:** Linear regression essentially minimizes the squared error distance between actual and predicted values. This was picked because of its simplicity and it would be interesting to gauge the performance of a classical regressor.

**Weakness:** With a smaller training set size, the RMSE is comparable to that of the other algorithms however once the training set increases, the RMSE gets much worse. This likely means that the algorithm performs poorly with extreme variation in the labels and the images they map to. This is not unexpected as linear models assume a linear relationship between independent variables and the dependent variables. This would not apply to facial positions especially if they are not all full frontal shots. Therefore, it is likely not reliable as an algorithm moving forward.

**Future:** There are other linear modeling techniques such as Lasso and ElasticNet which add regularization terms that often are used to reduce overfitting. The results would be interesting to see however the assumption of a linear relationship still applies. Also the choosing of the regularization parameter would warrant some deep investigation.

### Random Forest/Decision Tree

**Rational:** A decision tree is a non-parametric supervised learning method that predicts the value of a target variable by establishing decision rules inferred from the data features. Random forests are just a set of decision trees results where a consensus value has been established either by voting or averaging. We ran a random forest as well because of the weaknesses as described below. 

**Weakness:** By itself, this decision tree algorithm seemed to perform the worst after linear regression. It could be that we have overfitted the model since decisions can be very complex and is very sensitive to small variations. Tree are also known to ignore the correlation between features, which is highly the case in this experiment. For instance, knowing the position of one eye can often tell you the position of the other eye.

**Future:** Hyperparameter tuning to improve performance include pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth.

### KNN 

**Rational:** We pick kNN because all facial features are on the same location on the face i.e left eye is on the upper left, right eye is on the upper right, nose is in the middle, etc. Because kNN work by assigning a query point as those of its neighbors, given a set of images, kNN should be able to figure out which pixels is generally associated with the eye area, the nose area, etc.

**Weakness:** One potential weakness of this algorithm may be images where the face located off center. For example, if an image has a face translated to the right, then it may mistake the left eye on the image for a right eye

**Future:** One of the corner stones of knn is the idea of distance and how that is defined. We have found that Euclidean is better than the default Minknowski distance set by sklearn however the use of correlation might be more appropriate since as described above, the position of features on a face are all correlated in some sense. In other studies, Chi square distance also seems to perform well. Both of these would be worth engineering and exploring in the future. 

# Convolutional Neural Network

## Building the CNN

**Build a custom loss function to handle NA values**

In [ ]:
# building a time callback to track total runtime
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
# building a custom loss function to handle NAs
def mean_squared_error_with_missing(y_true, y_pred):
    
    # figure out which labels are present
    present = tf.logical_not(tf.is_nan(y_true))
    
    # select only those labels
    present_labels_true = tf.boolean_mask(y_true, present)
    present_labels_pred = tf.boolean_mask(y_pred, present)
    
    # compute mean squared error
    loss = tf.math.reduce_mean(tf.math.squared_difference(present_labels_true,present_labels_pred))
    
    return loss

**Build the model**

In [ ]:
mod = plt.imread('model.png')
fig = plt.figure(figsize=(10,20), frameon=False)
plt.imshow(mod)
plt.axis('off')

**Parameter and Hyper-parameter selection:**

The Convolutional Neural Network (CNN) has many parameters to choose to build the model. We first need to determine how many layer stacks the model should have. Each layer stack should contain, at a minimum, a convolutional layer and a pooling layer. You can also choose to add a dropout layer at each stack, to help avoid overfitting on the training set.  For each convolutional layer, we need to determine the shape of the convolutional kernel, as well the shape of the pooling kernel. After the convolutional layer stack, the output needs to be flattened and we need to determine the number of dense layers to send it through before generating the output. Again we can choose to add dropout layers here or not. After we have decided on the shape our network will take, we need to determine the optimizer to use, and the loss function. The optimizer comes with its own set of hyperparameters, the most important of which is the initial learning rate according to Bengio.

**Optimizer:**  
Per Bengio, for stochastic gradient descent (SGD) optimization, "The optimal learning rate is usually close to the largest learning rate that does not cause divergence of the training criterion." His suggestion is to start with a large learning rate and if the training criterion diverges, decrease the learning rate by a factor of 3. Due to hardware memory limitations, we were unable to carry out full hyperparameter tuning via grid search. So rather than search for the optimal SGD learning rate, we chose to use an adaptive optimizer called Adam which converges much faster than standard SGD. Adam is an adaptation/enhancement to SGD. Whereas SGD uses a single learning rate for all weight updates and does not update the learning rate during training, the Adam optimizer "calculates an exponential moving average of the gradient and the squared gradient." The hyperparameters beta1 and beta2 control the decay rates of these moving averages. Because the authors of the Adam optimizer performed hyperparameter tuning on a similar convolutional neural net built for prediction on the MNIST data set, we used the default values of the Adam optimizer which are currently set to the optimal values determined by the authors.

**Number of Layers:**  
Again according to Bengio, "the optimal number layers can be determine by simply adding layers until the error does not improve anymore." As we were using the Daniel Nouri tutorial as a guide to building the CNN, we started with three convolutional/pooling layer pairs in our CNN. We tested 4 and 5 layer pairs as well which gave little to no improvement in the loss on the validation data, but did increase the training time significantly, so we decided on three layer pairs for the default value of the model. 

**Number of filters:**  
Each filter in a convolutional layer can be thought of as a single feature detector. In the initial layer, the filters will be able to detect simple geometric patterns such as a lines or curves. As we go deeper in the network, the feature detectors can become more sophisticated. Since we want to predict the position of 15 facial keypoints varying in shape and size and orientation, we need to have enough feature detectors by the last layer to handle a large variety of features. We tried various combinations of factors of 2 for the number of filters in each layer, with the number of filters multiplying by 2 at each layer. After some trial and error, we settled on 32, 64, 128 filters for convolutional layers 1, 2, and 3 respectively. 

**Pooling layers:**  
The pooling layers determine the factor by which you downscale your data after each convolutional layer. We chose to simply use pooling kernels of (2,2) with a stride of (2,2) so that our output shape at each layer was easy to keep track of

**Number of Dense layers and Dense layer nodes:**  
Admittedly, we spent more of our time tuning the parameters in the convolutional stack than in the dense layer section of the model, but we did test some variations ranging from 1 layer with 100 nodes to 3 layers of 2000 nodes and ended up settling on 2 layers with 1000 nodes each for our default model values.


In [ ]:
def build_CNN(
    num_ConvLayers=3, 
    num_DenseLayers=2,
    filters=[32,64,128],
    kernels=[(3,3),(3,3),(3,3)], 
    pools=[(2,2),(2,2),(2,2)], 
    dense_nodes=[1000,1000], 
    optimizer='Adam', 
    dropout=None,
    input_shape=(96,96,1),
    loss=mean_squared_error_with_missing
):
    '''This function builds a CNN model
    num_ConvLayers --> is the number of convolutional layer groups where 
    each group contains a convolutional layer, a pooling layer, and an
    optional dropout layer
    
    num_DenseLayers --> is the number of dense layers after flattening
    the convolutional output
    
    filters --> a list of the number of filters used in each
    convolutional layer specified in num_ConvLayers
    
    kernels --> a list of the kernel size to use in each convolutional
    layer specified in num_ConvLayers
    
    pools --> a list of the kernel size to use in each pooling layer
    specified in num_ConvLayers
    
    dense_nodes --> a list of the number of units in each dense layer
    specified in num_DenseLayers
    
    dropout --> optional, a list of the dropout rates at each 
    convolutional layer
    
    input_shape --> the shape of the input data
    '''
    # define a Sequential model
    model = Sequential()
    
    # add convolutional layer groups
    for i in range(num_ConvLayers):
        model.add(Conv2D(
            filters=filters[i],
            kernel_size=kernels[i],
            strides=(1,1),
            padding='same',
            activation='relu',
            input_shape=input_shape
        ))
        model.add(MaxPooling2D(
            pool_size=pools[i],
            strides=None
        ))
        
        # add dropout layers if specified
        if dropout:
            model.add(Dropout(rate=dropout[i]))
    
    # flatten the convolutional layers
    model.add(Flatten())
    
    # add the dense layers
    for j in range(num_DenseLayers):
        model.add(Dense(
            units=dense_nodes[j],
            activation='relu'
        ))
        
    # create the output layer
    model.add(Dense(units=30))
    
    # compile the model
    model.compile(loss=loss, optimizer=optimizer)
    
    return model

**Train the model**

In [ ]:
def train_model(
    model, 
    train_data, train_labels, 
    dev_data=None, dev_labels=None,
    batch_size=32,
    verbose=1,
    epochs=10
):
    time_callback = TimeHistory()
    hist = keras.callbacks.History()
    if not dev_data is None:
        model.fit(
            train_data, train_labels, 
            batch_size=batch_size,
            epochs=epochs, 
            validation_data=(dev_data, dev_labels),
            verbose=verbose,
            callbacks=[hist, time_callback]
        )
    else:
        model.fit(
            train_data, train_labels, 
            batch_size=batch_size,
            epochs=epochs, 
            validation_split=0.2,
            verbose=verbose,
            callbacks=[hist, time_callback]
        )
    print('\nRMS error on validation set:', np.sqrt(hist.history['val_loss'][-1]) * 48)
    print('Total runtime (minutes):', round(np.sum(time_callback.times) / 60, 2))
    
    return hist, time_callback

###  Running CNN
the below table contains a layout of our training plan. The 0th model will be trained on the complete data set (no missing values) for performance comparison against the simple baseline models. The first true model will be trained on the original dataset with missing values and no augmentations applied. The second model will use the original dataset with all augmentations applied. The third model will use the augmented data with NAs filled by 2-layer NN predictions. The fourth model will be the same as model 3 but we will utilize dropout layers to prevent overfitting

<table> 
<tr>
<th> Model Name </th> <th> Model Parameters </th> <th> Data </th> <th> Training Criteria </th>
</tr>
<tr>
<td> Model 0a </td> <td> Default </td> <td> Complete Data </td> <td> 50 epochs </td>
</tr>
    <tr>
<td> Model 0b </td> <td> Default </td> <td> Complete Data + Augmentations </td> <td> 50 epochs </td>
</tr>
    <tr>
<td> Model 1 </td> <td> Default </td> <td> Original Data </td> <td> 50 epochs </td>
</tr>
    <tr>
<td> Model 2 </td> <td> Default </td> <td> Original Data + Augmentations </td> <td> 50 epochs </td>
</tr>
    <tr>
<td> Model 3 </td> <td> Default </td> <td> Aug. data, NAs filled by 2-layer NN predictions </td> <td> 50 epochs </td>
</tr>
    <tr>
<td> Model 4 </td> <td> Add dropout layers </td> <td> Aug. data, NAs filled by 2-layer NN predictions </td> <td> 50 epochs </td>
</tr>
    <tr>
</tr>
</table>

#### Model 0

In [ ]:
model0a = build_CNN()

In [ ]:
# train the model
hist0a, runtime0a = train_model(
    model0a, train_data_complete, train_labels_complete, 
    dev_data_complete, dev_labels_complete, epochs=50
)

# save off the history
history0a = pd.DataFrame(hist0a.history)
history0a.to_csv('history_model0a.csv')

# predict on test data and format for kaggle submission
pred0a = predict_for_kaggle(model0a, test_data, 'kaggle_model0a.csv')

# store the results
result_dict['CNN Model0a'] = [
    round(np.sqrt(hist0a.history['loss'][-1]) * 48,2), 
    round(np.sqrt(hist0a.history['val_loss'][-1]) * 48,2),
    round(np.sum(runtime0a.times),2)
]
df = pd.DataFrame(result_dict)
with open('results.csv','a') as f:
    df.to_csv(f, header=False)

In [ ]:
model0b = build_CNN()

In [ ]:
# train the model
hist0b, runtime0b = train_model(
    model0b, train_data_aug_comp, train_labels_aug_comp, 
    dev_data_complete, dev_labels_complete, epochs=50
)

# save off the history
history0b = pd.DataFrame(hist0b.history)
history0b.to_csv('history_model0b.csv')

# predict on test data and format for kaggle submission
pred0b = predict_for_kaggle(model0b, test_data, 'kaggle_model0b.csv')

# store the results
result_dict['CNN Model0b'] = [
    round(np.sqrt(hist0b.history['loss'][-1]) * 48,2), 
    round(np.sqrt(hist0b.history['val_loss'][-1]) * 48,2),
    round(np.sum(runtime0b.times),2)
]
df = pd.DataFrame(result_dict)
with open('results.csv','a') as f:
    df.to_csv(f, header=False)

#### Model 1

In [ ]:
model1 = build_CNN()

In [ ]:
# train the model
hist1, runtime1 = train_model(
    model1, train_data_orig, train_labels_orig, 
    dev_data_orig, dev_labels_orig, epochs=50
)

# save off the history
history1 = pd.DataFrame(hist1.history)
history1.to_csv('history_model1.csv')

# predict on test data and format for kaggle submission
pred1 = predict_for_kaggle(model1, test_data, 'kaggle_model1.csv')

# store the results
result_dict['CNN Model1'] = [
    round(np.sqrt(hist1.history['loss'][-1]) * 48,2), 
    round(np.sqrt(hist1.history['val_loss'][-1]) * 48,2),
    round(np.sum(runtime1.times),2)
]

We could try running this for longer, but the way the validation loss levels off even as the training loss continues to shrink shows that the model is overfitting. This is unsurprising given the small size of our data set, and the large number of parameters. For better results, we should next try the full (unaugmented) data set and see how it fares.

#### Model 2

In [ ]:
model2 = build_CNN()

In [ ]:
# train the model
hist2, runtime2 = train_model(
    model2, train_data_aug, train_labels_aug, 
    dev_data_orig, dev_labels_orig, epochs=50
)

# save off the history
history2 = pd.DataFrame(hist2.history)
history2.to_csv('history_model2.csv')

# predict on test data and format for kaggle submission
pred2 = predict_for_kaggle(model2, test_data, 'kaggle_model2.csv')

# store the results
result_dict['CNN Model2'] = [
    round(np.sqrt(hist2.history['loss'][-1]) * 48,2), 
    round(np.sqrt(hist2.history['val_loss'][-1]) * 48,2), 
    round(np.sum(runtime2.times),2)
]
df = pd.DataFrame(result_dict)
with open('results.csv','a') as f:
    df.to_csv(f, header=False)

This did significantly better and the validation loss was still improving alongside the training loss at the end of ten epochs. Given this continued improvement, it would be worth running this for longer.

### Predict missing values
A large percentage of the points are missing, meaning we are not using a large percentage of the information we have in the dataset. However facial keypoints will, in general, follow location patterns across images of faces. Uing a simple two layer neural net we use the moset commonly labeled key points to predict values for the features that are not labeled in a majority of the data. Then we can fill the missing labels in with these predicted points and retrain the data with no missing values

### Build the 2-layer NN

In [ ]:
basicNN = Sequential()
basicNN.add(Dense(units=1000,activation='relu'))
basicNN.add(Dense(units=1000,activation='relu'))
basicNN.add(Dense(units=1000,activation='relu'))

# create the output layer
basicNN.add(Dense(units=30))

# compile the model
basicNN.compile(loss=mean_squared_error_with_missing, optimizer='Adam')

In [ ]:
NN_hist, NN_time = train_model(
    basicNN,
    train_labels_aug_comp[:,[0,1,2,3,20,21,28,29]], train_labels_aug_comp,
    epochs=20
)

In [ ]:
idx = np.random.randint(0, len(train_data_aug), 16)
data_sample = train_data_aug[idx, :, :, :]

pred = basicNN.predict(train_labels_aug[:,[0,1,2,3,20,21,28,29]])

label_sample = pred[idx, :]
label_sample_true = train_labels_aug[idx, :]

plot_sample(data_sample, label_sample_true, labels2=label_sample)

In [ ]:
train_labels_full = train_labels_aug.copy()
for i in range(train_labels_full.shape[1]):
    for j,v in enumerate(train_labels_full[:,i]):
        if np.isnan(v):
            train_labels_full[j,i] = pred[j,i]

#### Model 3

In [ ]:
# augment some data or change a parameter
model3 = build_CNN()

Testing against augmented dev data with missing values

In [ ]:
# train the model
hist3, runtime3 = train_model(
    model3, train_data_aug, train_labels_full, 
    dev_data_orig, dev_labels_orig, epochs=50
)

# save off the history
history3 = pd.DataFrame(hist3.history)
history3.to_csv('history_model3.csv')

# predict on test data and format for kaggle submission
pred3 = predict_for_kaggle(model3, test_data, 'kaggle_model3.csv')

# store the results
result_dict['CNN Model3'] = [
    round(np.sqrt(hist3.history['loss'][-1]) * 48,2), 
    round(np.sqrt(hist3.history['val_loss'][-1]) * 48,2), 
    round(np.sum(runtime3.times),2)
]
df = pd.DataFrame(result_dict)
with open('results.csv','a') as f:
    df.to_csv(f, header=False)

#### Model 4

In this model, we add dropout layers after each pooling layer. The objective of the dropout layers is to help with generalization and avoid overfitting. The dropout layer randomly selects a percentage of nodes at each layer and throws their information out, prior to backpropagation. The idea is that some of the information you have "learned" is too specific too the training set and therefore throwing out a portion of it makes you learn slower, but helps you generalize better.

In [ ]:
# adding droput layers
model4 = build_CNN(dropout=[0.1,0.2,0.3])

Testing against augmented dev data with missing values

In [ ]:
# train the model
hist4, runtime4 = train_model(
    model4, train_data_aug, train_labels_full, 
    dev_data_orig, dev_labels_orig, epochs=50
)

# save off the history
history4 = pd.DataFrame(hist4.history)
history4.to_csv('history_model4.csv')

# predict on test data and format for kaggle submission
pred4 = predict_for_kaggle(model4, test_data, 'kaggle_model4.csv')
    
# store the results
result_dict['CNN Model4'] = [
    round(np.sqrt(hist4.history['loss'][-1]) * 48,2), 
    round(np.sqrt(hist4.history['val_loss'][-1]) * 48,2), 
    round(np.sum(runtime4.times),2)
]
df = pd.DataFrame(result_dict)
with open('results.csv','a') as f:
    df.to_csv(f, header=False)

#### Visualize training improvement
Doesn't look like an improvement, but it will once we train on the whole data sets

In [ ]:
hist0b = pd.read_csv('history_model0b.csv')
hist0a = pd.read_csv('history_model0a.csv')
hist1 = pd.read_csv('history_model1.csv')
hist2 = pd.read_csv('history_model2.csv')
hist3 = pd.read_csv('history_model3.csv')
hist4 = pd.read_csv('history_model4.csv')

In [ ]:
val_rms0a = hist0a['val_loss']
train_rms0a = hist0a['loss']
val_rms0b = hist0b['val_loss']
train_rms0b = hist0b['loss']
val_rms1 = hist1['val_loss']
train_rms1 = hist1['loss']
val_rms2 = hist2['val_loss']
train_rms2 = hist2['loss']
val_rms3 = hist3['val_loss']
train_rms3 = hist3['loss']
val_rms4 = hist4['val_loss']
train_rms4 = hist4['loss']

fig = plt.figure(figsize=(12,8))
plt.semilogy(val_rms0a, 'c-')
plt.semilogy(train_rms0a, 'c--')
plt.semilogy(val_rms0b, 'm-')
plt.semilogy(train_rms0b, 'm--')
plt.semilogy(val_rms1, 'g-')
plt.semilogy(train_rms1, 'g--')
plt.semilogy(val_rms2, 'b-')
plt.semilogy(train_rms2, 'b--')
plt.semilogy(val_rms3, 'r-')
plt.semilogy(train_rms3, 'r--')
plt.semilogy(val_rms4, 'k-')
plt.semilogy(train_rms4, 'k--')
plt.xlabel('Epochs')
plt.ylabel('Loss - Mean Squared Error')

plt.legend([
    'Complete Data - Val. Loss', 'Complete Data - Train Loss',
    'Complete Data + Aug. - Val. Loss', 'Complete Data + Aug. - Train Loss',
    'Orig. Data Only - Val. Loss', 'Orig. Data Only - Train Loss',
    'Orig. Data + Aug. - Val. Loss', 'Orig. Data + Aug. - Train Loss',
    'Orig. Data + Aug. + Pred. NAs - Val. Loss', 'Orig. Data + Aug. + Pred. NAs - Train Loss',
    'Add Dropout layers - Val. Loss', 'Add Dropout layers - Train Loss',
])

Here we are comparing the training performance of each of our CNN models. We tried running for longer than 50 epochs, but were limited by our hardware so we decided to cap it at 50 epochs. The dotted lines represent the training loss, and the solid lines represent the test loss. The lowest two lines represent the training loss on the complete data set with no missing values which is also being validated on the complete dev set with no missing values. You can see that we are starting to see pretty severe overfitting by the fact that the loss on the training data continues to trend downward, while the validation loss remains relatively flat. In general our models are converging fairly quickly, most of the time within the first 10 or 20 epochs. This is likely due to using the Adam optimizer which adapts the learning rate rapidly and converges much faster than traditional SGD. One notable exception is the fourth model where we added dropout layers and we can see the training loss converges much slower than the models with no dropout. There are two spikes that we see, the first blue spike we cannot explain, but the last spike in the dropout curve happened because our GPU ran out of memory and the training crashed.

# Visualize the Predictions

In [ ]:
# pull in the predicted data
LUT = pd.read_csv('IDLookupTable.csv')
p0a = pd.read_csv('kaggle_model0a.csv')
p0b = pd.read_csv('kaggle_model0b.csv')
p1 = pd.read_csv('kaggle_model1.csv')
p2 = pd.read_csv('kaggle_model2.csv')
p3 = pd.read_csv('kaggle_model3.csv')
p4 = pd.read_csv('kaggle_model4.csv')

# reformat the predicted data
LUT['p0a'] = p0a.Location
LUT['p0b'] = p0b.Location
LUT['p1'] = p1.Location
LUT['p2'] = p2.Location
LUT['p3'] = p3.Location
LUT['p4'] = p4.Location

cols = {'p0a','p0b','p1','p2','p3','p4'}
labels = [
    np.array([LUT[col][LUT.ImageId == i+1].values for i in range(6)])
    for col in cols
]

# plot a sample
idx = list(range(6))
sample_data = test_data[idx]

fig, axes = plt.subplots(6,6,figsize=(20,20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.2)
colors = ['c','m','g','b','r','y']
titles = ['net 0a', 'net 0b','net 1','net 2','net 3','net 4']
for i in range(6):
    for j in range(6):
        ax = axes[i,j]
        img = sample_data[j].reshape(96,96)
        labs = labels[i][j]
        ax.imshow(img, cmap='gray')
        ax.scatter(labs[::2], labs[1::2], c=colors[i])
        ax.set_title(titles[i], fontsize=20)
        ax.axis('off')

# Conclusion

Put a conclusion here comparing and contrasting the models and how well we did on Kaggle

## Summary Table

In [ ]:
# Baseline Models

summary = pd.DataFrame.from_dict(result_dict)
summary = summary.head(2)

kaggle_scores = pd.DataFrame([4.46, 4.08, 4.74, 3.91, 3.96, 46.32, 4.96, 4.11, 4.07], columns=summary.columns.values)

summary = summary.append(kaggle_scores, ignore_index=True)

summary

In [ ]:
# CNN Models

results = pd.read_csv('results.csv')
results.columns.values[0] = 'index'
results.set_index(results['index'], inplace=True)
results = results.iloc[:,1:]
results = results.head(3)

results

In [ ]:
results_table = pd.concat([results, summary], axis=1)
results_table 

# Sources

 - http://deeplearning.net/tutorial/lenet.html#lenet
 - https://arxiv.org/abs/1206.5533
 - https://arxiv.org/abs/1412.6980v8
 - http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/
 - https://arxiv.org/abs/1710.05279
 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4978658/

# Appendix

## Erroneous Data Algorithm

#### Note: It is not recommended to run this cell because it is out of sequence

During our EDA, we incidentally discovered that some data was very clearly mislabeled, so we decided to search for and remove bad data to reduce noise. Since the data set is large, we needed an algorithm to flag likely bad data. Our method of choice was to train our CNN model on the full training data, then use that model to predict values _on the training data_, flagging any image where a single keypoint was off by 40 pixels or more. We threw out 69 images that met this criterion. A few images looked mostly correct and may have been collateral damage. A further improvement attempt could be to only throw out the keypoints that were questionable and keep the ones that looked good. We did want to be wary of throwing out too much, since we were training on the same data we were predicting on, so our predictions were only so reliable. Another possible improvement could be to incorporate cross-validation within the training set for this error identification exercise.

In [ ]:
model1 = build_CNN()

time_callback = TimeHistory()
hist = keras.callbacks.History()

model1.fit(
    train_data_raw, train_labels_raw, 
    batch_size=32,
    epochs=1, 
    verbose=1,
    callbacks=[hist, time_callback]
)
predictions = model1.predict(train_data_raw, verbose=1)

# get array of squared errors
error = predictions*96 - train_labels_raw*96
sq_err = error ** 2

# flag any images where a single keypoint dimension is off by a threshold distance
flagged = []
for i in range(sq_err.shape[0]):
    threshold = 40
    if np.any(sq_err[i] > threshold ** 2):
        flagged.append(i)
print(flagged)

# plot these images as a sanity check that they are in face "bad" data
i = 0
rows = 8
cols = len(flagged)//rows + 1
fig = plt.figure(figsize=(20,20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for index in flagged:
    ax = fig.add_subplot(rows, cols, i + 1, xticks=[], yticks=[])
    ax.imshow(train_data_raw[index].reshape(96,96), cmap='gray')
    ax.scatter(train_labels_raw[index][0::2] * 48 + 48, train_labels_raw[i][1::2] * 48 + 48, c='r')
    i += 1